In [ ]:
%pylab
from glob import glob
from natsort import natsorted
import h5py
from palettable.cartocolors.qualitative import Bold_8, Bold_9, Bold_10
from natsort import natsorted as sorted

Q_over_m, L_over_m = 1/1.6e-47, 1067.8216118445728  # IMF-averaged specific luminosity and photon production rate

## Figure 2

In [5]:
fig, ax = plt.subplots(1,1,figsize=(4,4))
ax2 = ax.twiny()

rho0 = (2e4 * 3 / (4*np.pi*10**3))
tff = (3*np.pi / (32 * 4300.7 * rho0))**0.5 * 979
ax2.set(xlim=[0,10/tff],xlabel=r"$t/t_{\rm ff,0}$",xticks=[0,1,2])
ax.set_prop_cycle('color', Bold_9.mpl_colors)
times,r50, rho_avg,rho50,sigma_3D, fmag, alpha, mbound,mmol,mhii,rad,fmag = np.loadtxt("global_gas_props.dat")[::10].T
times *= 979
ax.plot(times,r50,label=r"$R_{\rm eff}\,\left(\rm pc\right)$",lw=1.5)
ax.plot(times,rho50,label=r"$n_{\rm H}^{\rm eff}\,\left(\rm cm^{-3}\right)$",lw=1.5)
ax.plot(times,rad,label=r"$e_{\rm rad}^{\rm eff} \,\left(\rm eV \, cm^{-3}\right)$",lw=1.5)
ax.plot(times,sigma_3D,label=r"$\sigma_{\rm 3D} \,\left(\rm km\,s^{-1}\right)$",lw=1.5)
ax.plot(times,alpha,label=r"$\alpha_{\rm turb} $",lw=1.5)
ax.plot(times[1:],mmol[1:],label=r"$M_{\rm H_{\rm 2}}\,\left(M_\odot\right)$",lw=1.5,ls='dashed')#,label=r"$\alpha_{\rm turb} $")
ax.plot(times,mhii,label=r"$M_{\rm HII}\,\left(M_\odot\right)$",lw=1.5,ls='dashed')
ax.plot(times,mbound,label=r"$M_{\rm bound}\,\left(M_\odot\right)$",lw=1.5,ls='dashed')
ax.plot(times,fmag,label=r"$E_{\rm mag}/E_{\rm turb}$",lw=1.5,ls='dotted')

t_SN = 0.0087*979
ax.plot([t_SN,t_SN],[0,1e37],ls='dotted',color='black')
ax.text(t_SN*1.01,1e4,"1st SN",rotation=90)

ax.set(xlabel="Time (Myr)",yscale='log',ylim=[0.01,4e6],xlim=[0,10])
ax.legend(labelspacing=0.1,fontsize=8,ncol=2,loc=2)
plt.savefig("global_gas_props.pdf",bbox_inches='tight')

## Figure 3

In [7]:
times,rho_stargas, rho_starstar, mstar, r50_star, lum, lum_MS, lum_acc, Q, F_jets, L_jets, F_wind, L_wind, sigma_3D_stars, vr_stars, Nstars, Mstar_max, Mstar_med, Mstar_mean = np.loadtxt("global_star_props.dat")[::10].T
fig, ax = plt.subplots(1,1,figsize=(4,4))
ax2 = ax.twiny()

rho0 = (2e4 * 3 / (4*np.pi*10**3))
times = np.array(times)*979
tff = (3*np.pi / (32 * 4300.7 * rho0))**0.5 * 979
ax2.set(xlim=[0,10/tff],xlabel=r"$t/t_{\rm ff,0}$",xticks=[0,1,2])
ax.set_prop_cycle('color', Bold_10.mpl_colors)
ax.plot(times, mstar, label=r"$M_{\rm \star}^{\rm tot}\,(M_\odot)$",lw=1.5)

ax.plot(times, Nstars, label=r"$N_{\rm \star}$",lw=1.5,ls='dashed')
t_nstar = np.interp(0.5, Nstars/Nstars.max(), times)
ax.plot([t_nstar,t_nstar],[0,1e100],color='black',ls='dashed',zorder=-100)
ax.text(t_nstar*1.02, 3e3,"50\% of stars",rotation=90)
t_mstar = np.interp(0.5, mstar/mstar.max(), times)
ax.plot([t_mstar,t_mstar],[0,1e100],color='black',ls='dotted',zorder=-100)
ax.text(t_mstar*1.02, 3e3,"50\% of mass",rotation=90)

L_acc = lum-lum_MS

ax.plot(times, rho_starstar,lw=1.5,label=r"$\tilde{\rho}_{\rm \star}^{\rm NN}\,(M_\odot\rm \,pc^{-3})$",ls='dashdot')
ax.plot(times, r50_star,lw=1.5,label=r"$R_{\rm \star}^{\rm eff}\,\rm (pc)$")
ax.plot(times, sigma_3D_stars/1e3,lw=1.5,label=r"$\sigma_{\rm \star}\,(\rm km\,s^{-1})$")

rho50 = 0.5*mstar / (4*np.pi/3 * r50_star**3)
ax.plot(times,rho50,lw=1.5,label=r"$\rho_{\rm \star}^{\rm eff}\,(M_\odot\rm \,pc^{-3})$",ls='dashdot')

ax.plot(times,vr_stars/1e3,lw=1.5,label=r"$|\tilde{v}_{\rm r}|\,(\rm km\,s^{-1})$",color='cornflowerblue')
ax.plot(times,-vr_stars/1e3,lw=1.5,color='cornflowerblue',ls='dashed')

ax.legend(labelspacing=0.1,fontsize=8,ncol=1,handletextpad=0.2)

ax.set(xlabel="Time (Myr)",yscale='log',ylim=[0.1,1e5],xlim=[0,10])
plt.savefig("global_star_props.pdf",bbox_inches='tight')

## Figure 5

In [9]:
data = np.loadtxt("global_star_props.dat")
stride = 10
data = np.diff(data.cumsum(axis=0)[::stride],axis=0)/stride

times,rho_stargas, rho_starstar, mstar, r50_star, lum, lum_MS, lum_acc, Q, F_jets, L_jets, F_wind, L_wind, sigma_3D_stars, vr_stars, Nstars, Mstar_max, Mstar_med, Mstar_mean = data.T
fig, ax = plt.subplots(1,1,figsize=(4,4))
ax2 = ax.twiny()

rho0 = (2e4 * 3 / (4*np.pi*10**3))
times = np.array(times)*979
tff = (3*np.pi / (32 * 4300.7 * rho0))**0.5 * 979
ax2.set(xlim=[0,10/tff],xlabel=r"$t/t_{\rm ff,0}$",xticks=[0,1,2])
from palettable.cartocolors.qualitative import Bold_6
cmap = Bold_6
ax.set_prop_cycle('color', cmap.mpl_colors)
L_acc = lum_acc

ax.plot(times, lum, label=r"$L_{\rm tot}\,(L_\odot)$",lw=1.5,zorder=100,ls='dashed')
ax.plot(times, L_acc, label=r"$L_{\rm acc}\,(L_\odot)$",lw=1.5,ls='dashed')
ax.plot(times, lum-L_acc, label=r"$L_{\rm fus}\,(L_\odot)$",lw=1.5,ls='dashed')
ax.plot(times, L_wind, label=r"$\dot{E}_{\rm wind}\,(L_\odot)$",lw=1.5)
ax.plot(times, L_jets, label=r"$\dot{E}_{\rm jets}\,\left(L_\odot\right) $",lw=1.5)
ax.plot(times, Q/1e47, label=r"$\mathcal{Q} \,\left(10^{47} \rm s^{-1}\right) $",lw=1.5)

ax.plot(times, mstar*L_over_m, label=r"$M_{\rm \star}^{\rm tot}  \langle L/M_{\rm \star} \rangle \,(L_\odot)$",lw=1.5,ls='dotted',zorder=-1,color=cmap.mpl_colors[0])
ax.plot(times, mstar*Q_over_m/1e47, label=r"$M_{\rm \star}^{\rm tot} \langle \mathcal{Q}/M_{\rm \star}\rangle \,(10^{47} s^{-1})$",lw=1.5,ls='dotted',zorder=-1,color=cmap.mpl_colors[-1])

ax.plot([0,10],[8.5,8.5],ls='dashed',color='black')
ax.text(5,4,r"$G^{3/2} M_{\rm 0}^{5/2} R_{\rm 0}^{-5/2}$ ($L_\odot$)",fontsize=6)
t_SN = 0.0087*979
ax.plot([t_SN,t_SN],[0,1e37],ls='dotted',color='black')
ax.text(t_SN*1.01,1e7,"1st SN",rotation=90)
ax.legend(labelspacing=0,fontsize=8,loc=2,handletextpad=0.2,frameon=False,framealpha=0.5,ncol=2)

ax.set(xlabel="Time (Myr)",yscale='log',ylim=[1,1e8],xlim=[0,10],yticks=np.logspace(0,8,9))#,ylim=[0.1,1e5],xlim=[0,8])
plt.savefig("global_fb_energy_props.pdf",bbox_inches='tight')

## Figure 6

In [43]:
data = np.loadtxt("global_star_props.dat")
stride = 10
data = np.diff(data.cumsum(axis=0)[::stride],axis=0)/stride

times,rho_stargas, rho_starstar, mstar, r50_star, lum, lum_MS, lum_acc, Q, F_jets, L_jets, F_wind, L_wind, sigma_3D_stars, vr_stars, Nstars, Mstar_max, Mstar_med, Mstar_mean = data.T
fig, ax = plt.subplots(1,1,figsize=(4,4))
ax2 = ax.twiny()

rho0 = (2e4 * 3 / (4*np.pi*10**3))
times = np.array(times)*979
tff = (3*np.pi / (32 * 4300.7 * rho0))**0.5 * 979
ax2.set(xlim=[0,10/tff],xlabel=r"$t/t_{\rm ff,0}$",xticks=[0,1,2])
from palettable.cartocolors.qualitative import Bold_4
ax.set_prop_cycle('color', Bold_4.mpl_colors)
L_acc = lum_acc #np.clip(lum - lum_MS,0,1e100)

ax.plot([0,10], np.ones(2)* 2e4**2 / 10**2 * 0.21,ls='dashed',color='black')#,label=r"$GM_{\rm 0}^2/R_{\rm 0}^2$ ($L_\odot/c$)",color='black')
ax.text(0.4,0.6e6,r"$GM_{\rm 0}^2/R_{\rm 0}^2$ ($L_\odot/c$)",fontsize=6)
ax.plot(times, lum, color='black',lw=1.5)
ax.plot(times, lum, label=r"$L_{\rm tot}/c\,(L_\odot/c)$",lw=1)
ax.plot(times, F_wind, color='black',lw=1.5)
ax.plot(times, F_wind, label=r"$\dot{P}_{\rm wind} \,\left(L_\odot/c\right)$",lw=1)
ax.plot(times, F_jets, color='black',lw=1.5)
ax.plot(times, F_jets, label=r"$\dot{P}_{\rm jets}\,\left(L_\odot/c\right) $",lw=1)

t_SN = 0.0087*979
ax.plot([t_SN,t_SN],[0,1e37],ls='dotted',color='black')
ax.text(t_SN*1.01,3e6,"1st SN",rotation=90)
#ax.plot(times,np.abs(vr_stars))
ax.legend(labelspacing=0,fontsize=8,loc=2,handletextpad=0.2,frameon=False,framealpha=0.5,ncol=1)

ax.set(xlabel="Time (Myr)",yscale='log',ylim=[1e3,1e7],xlim=[0,10])#,ylim=[0.1,1e5],xlim=[0,8])
plt.savefig("global_fb_mom_props.pdf",bbox_inches='tight')